# TF-IDF
Recapitulando un poco nuestro proyecto, queremos que dada una subcadena encontrar algun subtitulo que contenga dicha subcadena. 
TF-IDF nos permite buscar entre una gran cantidad de documentos a diferencia de RAKE. Además nos permite obtener un ejemplo mas representativo de dicha subcadena, pues TF-IDF le da un valor a la representatividad.
TF se encarga de encontrar la frecuencia de palabras dentro de un documento e IDF se encarga de revisar las estas frecuencias y avergiuar que tan representativas son para cada documento (video).

Si acotamos el problema a meramente coincidencias de las palabras no necesitamos ningun espacio vectorial porque no nos interesa la semantica de las palabras. Pues queremos coincidencias exactas y no frases con una semantica similar.

Tambien, si decidieramos implementar un espacio vectorial, podriamos hacer uso de vectores TF-IDF. Tomando en cuenta que nuestros documentos seran nuestros subtitulos en lugar de tomar cada video como un documento.

In [43]:
import pickle
from tqdm import tqdm 
from collections import defaultdict, Counter
import pandas as pd
from operator import itemgetter
import numpy as np

In [5]:
with open("../clean_videos.pkl", "rb") as f:
    videos = pickle.load(f)
videos[0][0]

{'id': 'L9YhoRatRzE',
 'original_title': 'Siempre Fui Yo | Adelanto | Disney+',
 'subtitles': [{'start': '0.13',
   'dur': '3.77',
   'text': ['tu', 'papá', 'tuvo', 'un', 'accidente']},
  {'start': '12.5', 'dur': '5.939', 'text': ['te', 'recuerdo', 'que', 'está']},
  {'start': '15.59', 'dur': '5.339', 'text': ['aquí']},
  {'start': '18.439',
   'dur': '6.361',
   'text': ['estaba', 'como', 'rabioso', 'con']},
  {'start': '20.929',
   'dur': '7.65',
   'text': ['especial', 'con', 'lucas', 'martín']},
  {'start': '24.8',
   'dur': '3.779',
   'text': ['necesito', 'saber', 'qué', 'fue', 'lo', 'que', 'pasó']},
  {'start': '29.42', 'dur': '2.479', 'text': ['aplausos']}]}

In [58]:
documents = defaultdict(list)
def merge_videos(corpus):
    '''
    Mezcla el corpus de cada video en una entrada del
    diccionario movies en un solo corpus, 
    guarda los diálogos por línea tokenizados y limpios

    Args:
        col (dic): Diccionario con los diálogos
    '''
    corpus_clean = []
    #Iteramos sobre los canales
    for chanel in tqdm(corpus):
        for video in chanel:
            #Cada texto se guarda por oraciones
            if 'subtitles' in video:
                corpus_clean += [s['text'] for s in video['subtitles']]
                for s in video['subtitles']:
                    documents[video['id']] += s["text"]
    return corpus_clean

all_videos = merge_videos(videos)
all_videos[0]

100%|████████████████████████████████████████████| 9/9 [00:00<00:00, 23.90it/s]


['tu', 'papá', 'tuvo', 'un', 'accidente']

In [15]:
def flatten(lst):
    return [item for sub in lst for item in sub]

In [22]:
term_frequencies = Counter(flatten(all_videos))

In [25]:
term_list = pd.DataFrame(sorted(term_frequencies.items(), key=itemgetter(1), reverse=True), 
                         columns=['Token','Frequency'])

term_list = term_list.set_index(term_list['Token'])
term_list.pop('Token')
term_list

,Frequency
Token,
de,217897
que,167644
la,128609
y,116389
el,105011
...,...
nananana,1
nía,1
llull,1


In [68]:
idf = {}
num_documents = len(documents)
for term in tqdm(term_list.index):
    #Total de documentos en que
    #aparece el término
    total_documents = 0
    for doc, token_list in documents.items():
        if term in token_list:
            total_documents += 1
            
    #Asignación de idf
    #print(total_documents, num_documents)
    idf[term] = -np.log2(total_documents/num_documents)
    
pickle.dump(idf, open("../idf.pkl", "wb"))

#Agregamos el idf al dataframe
term_list['idf'] = idf.values()
term_list.sort_values(by='idf', ascending=False)
    
pickle.dump(term_list, open("../term_list.pkl", "wb"))

100%|██████████████████████████████████| 97791/97791 [3:54:51<00:00,  6.94it/s]


In [65]:
{"a":1}

{'a': 1}

In [61]:
documents["L9YhoRatRzE"]

['tu',
 'papá',
 'tuvo',
 'un',
 'accidente',
 'te',
 'recuerdo',
 'que',
 'está',
 'aquí',
 'estaba',
 'como',
 'rabioso',
 'con',
 'especial',
 'con',
 'lucas',
 'martín',
 'necesito',
 'saber',
 'qué',
 'fue',
 'lo',
 'que',
 'pasó',
 'aplausos']